In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from number_of_experimetns import calculate_number_of_replicas
from experimenter_knative import experiment_with
sns.set_theme()

In [ ]:
# Maximum number of replicas that the system can manage.
MAX_REPLICA_COUNT = 10_000

# Number of experiments
NUMBER_OF_EXPERIMENTS = 9

# Each experiment will be done by this many times for consistency.
EXPERIMENT_PROOF = 7

# Int array of replicas for each experiment
REPLICAS: np.ndarray = calculate_number_of_replicas(MAX_REPLICA_COUNT, NUMBER_OF_EXPERIMENTS)

# How much should we wait for the kubernetes api to be stabilized
COOLDOWNS: np.ndarray = 10 * np.ones((NUMBER_OF_EXPERIMENTS,), dtype=int) + (REPLICAS * 0.004).astype(int)
# Namespace of the experimentation
NAMESPACE = "experiments"

In [ ]:
def draw_replicas(replicas=REPLICAS):
    plt.figure(figsize=(8, 4))
    plt.plot(range(len(replicas)), replicas)
    plt.title("Replica Count by Experiments")
    plt.xlabel("Experiment")
    plt.ylabel("Replicas")
    plt.show()

draw_replicas()
print(COOLDOWNS)
print(REPLICAS)

In [ ]:
# Do the experiment
experiment_with(REPLICAS.tolist(), NAMESPACE, COOLDOWNS.tolist(), EXPERIMENT_PROOF)